In [6]:
import numpy as np
import pandas as pd
from marcap import marcap_data
from datetime import datetime, timedelta

In [4]:
# 백테스트 기간의 가격 정보를 가져온다.
df_price = marcap_data('2000-01-01', '2019-12-31')

In [65]:
quarter_start_month = [1, 4, 7, 10]
days = []
year = 2002
for i in range(10):
    month = quarter_start_month[i % 4]
    start_date = datetime(year, month, 1)
    
    if month == 10:
        year += 1
    
    month = quarter_start_month[(i + 1) % 4]
    end_date = datetime(year, month, 1) - timedelta(days=1)
    
    day = {'start_date':start_date, 'end_date':end_date}
    days.append(day)

In [66]:
df_account = pd.DataFrame(columns=['start_date', 'end_date'])

In [67]:
df_account = df_account.append(days)

In [68]:
df_account

,start_date,end_date
0,2002-01-01,2002-03-31
1,2002-04-01,2002-06-30
2,2002-07-01,2002-09-30
3,2002-10-01,2002-12-31
4,2003-01-01,2003-03-31
5,2003-04-01,2003-06-30
6,2003-07-01,2003-09-30
7,2003-10-01,2003-12-31
8,2004-01-01,2004-03-31
9,2004-04-01,2004-06-30


In [72]:
df_price.head()

,Code,Name,Close,Changes,ChagesRatio,Volume,Amount,Open,High,Low,Marcap,MarcapRatio,Stocks,ForeignShares,ForeignRatio,Rank
Date,,,,,,,,,,,,,,,,
2000-01-04,030200,한국통신공사,169000.0,-10000.0,-5.6,2344578.0,4.020993e+11,182000.0,186000.0,160000.0,5.276174e+13,54.91,312199659.0,15609982.0,5.00,1.0
2000-01-04,005930,삼성전자,305500.0,39500.0,14.8,1483967.0,4.442332e+11,300000.0,305500.0,283000.0,4.577575e+13,47.64,149838777.0,70856428.0,47.29,2.0
2000-01-04,032390,한통프리텔,250500.0,-34000.0,-12.0,226168.0,5.665508e+10,250500.0,250500.0,250500.0,3.295369e+13,34.29,131551666.0,12783466.0,9.72,3.0
2000-01-04,017670,SK텔레콤,3800000.0,-270000.0,-6.6,35998.0,1.355300e+11,3901000.0,4050000.0,3470000.0,3.167594e+13,32.96,8335775.0,2636340.0,31.63,4.0
2000-01-04,015760,한국전력,36000.0,800.0,2.3,2341020.0,8.519245e+10,35600.0,38500.0,34550.0,2.303608e+13,23.97,639891145.0,143173933.0,22.37,5.0


In [73]:
start_date

datetime.datetime(2004, 4, 1, 0, 0)

In [76]:
year_start_date = start_date - timedelta(days=365)
year_end_date = start_date

candidate_price = df_price[(df_price.index > year_start_date) & (df_price.index <= year_end_date)]

In [80]:
len(candidate_price)

411384

In [69]:
def get_stocks(start_date):
    year_start_date = start_date - timedelta(days=365)
    year_end_date = start_date
    
    candidate_price = df_price[(df_price.index > year_start_date) & (df_price.index <= year_end_date)]
    
    return []

In [70]:
# 연도별 종목을 구한다.
df_account['stocks'] = df_account.apply(lambda x: get_stocks(x.start_date), axis=1)

2002-01-01 00:00:00
2002-04-01 00:00:00
2002-07-01 00:00:00
2002-10-01 00:00:00
2003-01-01 00:00:00
2003-04-01 00:00:00
2003-07-01 00:00:00
2003-10-01 00:00:00
2004-01-01 00:00:00
2004-04-01 00:00:00


In [71]:
df_account

,start_date,end_date,stocks
0,2002-01-01,2002-03-31,[]
1,2002-04-01,2002-06-30,[]
2,2002-07-01,2002-09-30,[]
3,2002-10-01,2002-12-31,[]
4,2003-01-01,2003-03-31,[]
5,2003-04-01,2003-06-30,[]
6,2003-07-01,2003-09-30,[]
7,2003-10-01,2003-12-31,[]
8,2004-01-01,2004-03-31,[]
9,2004-04-01,2004-06-30,[]
